In [164]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [18]:
#GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [93]:
# Create the data
a = np.round(  np.linspace(-10,10,num=10000)  )
np.random.shuffle(a)

b = np.round(  np.linspace(-10,10,num=10000)  )
np.random.shuffle(b)

sumAB = np.add(a, b)

In [125]:
# Normalize the data
aNorm    = a / np.max(a)
bNorm    = b / np.max(b)
sumNorm  = sumAB / np.max(sumAB)
dataNorm = np.vstack((aNorm, bNorm)).T

In [127]:
dataNorm.shape

(10000, 2)

In [128]:
sumNorm.shape

(10000,)

In [131]:
# Step1: Convert numpy data into tensor data
x = torch.tensor( dataNorm ).short()
y  = torch.tensor( sumNorm,  ).short()

In [148]:
# Step2: use scikit learn to split the data
train_data, test_data, train_labels, test_labels = train_test_split(x, y, test_size=0.1)

In [149]:
# Step 3: Convert into PyTorch Datasets
train_data = TensorDataset(train_data, train_labels)
test_data  = TensorDataset(test_data , test_labels)

In [155]:
train_data.tensors[0].shape

torch.Size([9000, 2])

In [156]:
test_data.tensors[0].shape

torch.Size([1000, 2])

In [150]:
# Step 4: Translate into dataloader objects
batchsize    = 32
train_loader = DataLoader(train_data, batch_size=batchsize, shuffle=True, drop_last=True)
test_loader  = DataLoader(test_data,  batch_size=test_data.tensors[0].shape[0])

In [165]:
# Create the Deep Learning Model
def createTheSumABNet():
    class sumABNet(nn.Module):
        def __init__(self):
            super().__init__()
            
            self.input = nn.Linear(2,20)
            
            self.bnormHidden =  nn.BatchNorm1d(20)
            self.hidden1 = nn.Linear(20,10)
            
            self.output = nn.Linear(10,1)
            
        def forward(self, x):
            
            x = F.leaky_relu( self.input(x) )
            
            x = self.bnormHidden(x)
            x = F.leaky_relu( self.hidden1(x) )
                        
            x = self.output(x)
            return x
        
    sumNet = sumABNet()
    
    lossfun = nn.MSELoss()
    
    optimizer = torch.optim.Adam(sumNet.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-8)
    
    return sumNet, lossfun, optimizer

In [178]:
# test the model that is has NO ERRORS!

sumNet, lossfun, optimizer = createTheSumABNet()

input = torch.rand(1,2)
sumNet.eval()
sumNet(input)
print(input)
sumNet.train()

tensor([[0.8933, 0.2738]])


sumABNet(
  (input): Linear(in_features=2, out_features=20, bias=True)
  (bnormHidden): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (hidden1): Linear(in_features=20, out_features=10, bias=True)
  (output): Linear(in_features=10, out_features=1, bias=True)
)

In [180]:
# Train the Model

def trainTheModel():
    
    #number of epochs to train
    numepochs = 50
    
    #model instance
    netSum, lossfun, optimizer = createTheSumABNet()
    
    #initialize losses & accuracy
    losses   = torch.zeros(numepochs)
    trainAcc = []
    testAcc  = []
    
    for epochi in range(numepochs):
        
        #batch loss & accuracy
        batchLoss = []
        batchAcc  = []
        
        #loop over mini-batches
        for X,y in train_data:
            
            #Forward pass & loss
            yHat = netSum(X)
            loss = lossfun(yHat, y)
            
            #backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #loss from this batch
            batchLoss.append(loss.item())
            
            #accuracy from this batch
            matches = torch.argmax(yHat,axis=1) == y ##????????????????
            matchesNumeric = matches.float()
            batchAcc.append( 100*torch.mean(matchesNumeric) )
        
        #average accuracy across mini-batches
        trainAcc.append( np.mean(batchAcc) )
        
        #average losses across all mini-batches
        losses[epochi] = np.mean(batchLoss)
        
        #Test Accuracy
        X,y = next(iter(test_data))
        with torch.no_grad():
            yHat = netSum(X)
            
        #compute the test accuracy
        testMatches = torch.argmax(yHat,axis=1) == y
        testMatchesNumeric = testMatches.float()
        testAcc.append(100 * torch.mean(testMatchesNumeric))
    
    return trainAcc, testAcc, losses, netSum

In [181]:
# Test the training code that it has NO ERRORS
trainAcc, testAcc, losses, netSum = trainTheModel()


RuntimeError: mat1 and mat2 must have the same dtype, but got Short and Float